# Introduction

[insert intro]


In [ ]:
%pip install pandas
%pip install matplotlib

In [ ]:
import math
import pandas as pd
import matplotlib.pyplot as plt

# Constants
g = 9.81                        # gravity [m/s^2]
rho = 1000                      # density of water [kg/m^3]
mu = 0.001                      # dynamic viscosity [Pa·s]
kv = 1.004e-6                   # kinematic viscosity [m^2/s]

eps = 1.5e-6                    # pipe roughness [m] (smooth plastic)
D = 0.00794                     # pipe diameter [m]
A = math.pi * (D / 2) ** 2      # pipe cross-sectional area [m^2]
tank_length = 0.32              # [m]
tank_width = 0.26               # [m]
A_t = tank_length * tank_width  # [m^2]
tolerance = 0.001               # tolerance to converge within
K = 0.5                         # minor losses

In [ ]:
# Compute drain time for given L and K
def compute_drain_time(L):
    # Height values
    h0 = 0.1 + L/150     # initial height [m]
    hf = 0.02 + L/150    # final height [m]
    sqrt_h_term = math.sqrt(h0) - math.sqrt(hf)

    f0 = 0.014
    f1 = 0.0
    v = 0.0
    Re = 0.0

    # Iterate until f converges
    while True:
        # Calculate velocity v (m/s)
        v = math.sqrt(2 * g * h0 / (1 + f0 * L / D + K))

        # Calculate Reynolds number
        Re = v * D / kv

        # Calculate friction factor based on flow regime
        if Re < 2300:
            # Laminar flow
            f1 = 64 / Re
        elif 2300 <= Re <= 4000:
            # Transitional flow: average laminar and turbulent f
            f_lam = 64 / Re
            # Colebrook-White turbulent friction factor estimate
            f_turb = 0.25 / (math.log10(eps / (3.7 * D) + 5.74 / (Re ** 0.9))) ** 2
            f1 = (f_lam + f_turb) / 2
        else:
            # Turbulent flow (Colebrook-White formula approximation)
            f1 = 0.25 / (math.log10(eps / (3.7 * D) + 5.74 / (Re ** 0.9))) ** 2

        # Check convergence
        if abs(f1 - f0) < 0.001:
            break

        f0 = f1


    T = (2 * A_t / A) * math.sqrt((1 + f0 * L / D + K) / (2 * g)) * sqrt_h_term
    return round(T, 2), round(v, 3), int(Re), round(f0, 5)

In [ ]:
cases = [
    {"label": "L = 20 cm", "L": 0.20},
    {"label": "L = 30 cm", "L": 0.30},
    {"label": "L = 40 cm", "L": 0.40},
    {"label": "L = 60 cm", "L": 0.60},
    # {"label": "L = 40 cm + T-joint", "L": 0.40},
]

# Run and collect results
results = []
for case in cases:
    T, v, Re, f = compute_drain_time(case["L"])
    results.append({
        "Case": case["label"],
        "Length (m)": case["L"],
        "Drain Time (s)": T,
        "Velocity (m/s)": v,
        "Re": Re,
        "f": f
    })

# Convert to DataFrame
df = pd.DataFrame(results)
display(df)


# Conclusion

[insert results and conclusion]
